# Alpha Range Check
THe range for the 8 fitting parameters seem to fluctuate between rounds. Some are exceptionally different than the rest. This notebook should print out the min/max ranges for all the alphas in all the rounds.

## Notes
## Alpha0
This is the bias term and will respond to any multipliers in the data. The other alphas however, do not. Which means, if there was an unaccounted system wide gain, alpha0 is the only one which would be affected.

## Settings
It might be helpful if there was a way to pull the settings(txt) file/additional information associated with each round. (Maybe later)

### Extreme Outliers
For 4 rounds, the alpha values are way way off. All of them are from sp2021 and 3/4 of them are 4 detector setups.

In [12]:
from inverse_modelling_tfo.data.intensity_interpolation import interpolate_exp_chunk, get_interpolate_fit_params
from typing import List, Union
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from TFO_dataset import SheepData

In [13]:
def prepare_patient_ppg(ppg_data : pd.DataFrame, sample_number : Union[int, List], SDD = [15, 30, 45, 70, 100], weights=[1.0, -1.0]) -> np.ndarray:
    """Create fitting parameters from PPG data. 

    Args:
        ppg_data (pd.DataFrame): PPG data DataFrame. You can feed data directly from the the TFO_dataset package.
        (Note: This should ideally be the optically normalized data)
        sample_number (int): which sample to choose. You can either pass a single integer or an array
        SDD (_type_, optional): Detector distances in TFO device(in mm). Defaults to SDD=[15, 30, 45, 70, 100].
        weights: Exponential weights applied during fitting to the detectors
    """
    # The code is generalized to run on any array. make necessary conversions 
    if isinstance(sample_number, int):
        sample_number = [sample_number]
    
    patient_features = []
    for sample_point in sample_number:
        # Pick a point in time
        spatial_intensity = ppg_data.iloc[sample_point]
        spatial_intensity *=  np.pi * 4   # Normalize to simulation detector shape
        # Reshape ppg data to fit the format
        spatial_intensity_wv1 = pd.DataFrame(data={
            'SDD' : SDD,
            'Intensity' : spatial_intensity.to_numpy()[:len(SDD)]
        })
        spatial_intensity_wv2 = pd.DataFrame(data={
            'SDD' : SDD,
            'Intensity' : spatial_intensity.to_numpy()[5: 5 + len(SDD)]
        })
        alpha_wv1 = interpolate_exp_chunk(spatial_intensity_wv1, weights=weights, return_alpha=True).flatten()
        alpha_wv2 = interpolate_exp_chunk(spatial_intensity_wv2, weights=weights, return_alpha=True).flatten()
        patient_features.append([alpha_wv1[0], alpha_wv2[0], alpha_wv1[1], alpha_wv2[1], alpha_wv1[2], alpha_wv2[2], alpha_wv1[3], alpha_wv2[3]])
        
    return np.array(patient_features)

In [14]:
sample_spacing = 1000     # Choosing all points would be very slow
shd = SheepData('iq_demod_optical')
all_tags = shd.get_available_tags()

print(len(all_tags))

comparison_table = pd.DataFrame(columns=['Sheep', 'Round', 'Year', 'Info', 'a0_min', 'a0_max', 'a1_min', 'a1_max', 'a2_min', 'a2_max', 'a3_min', 'a3_max'])
for tag_index, tag in enumerate(all_tags):
    # Not all rounds have avaialble normalizations constants/ignore those
    try:
        data = shd.get_data_from_tag(tag)
        sdd = shd.get_sdd_distance(tag)
    except:
        continue
    new_row = {'Sheep' : tag['experiment_number'], 'Round' : tag['experiment_round'], 'Year': tag['experiment_year_prefix'], 
               'Info': tag['additional_info']}
    features = prepare_patient_ppg(data, np.arange(0, len(data), sample_spacing), sdd, [1, 0.5])
    for feature_index in list(range(0, 8, 2)):  # Only wv1 features
        feature_max = np.nanmax(features[:, feature_index])
        feature_min = np.nanmin(features[:, feature_index])
        new_row[f'a{feature_index//2}_min'] = feature_min
        new_row[f'a{feature_index//2}_max'] = feature_max
    new_row = pd.DataFrame(new_row, index=[tag_index])
    comparison_table = pd.concat([comparison_table, new_row])

pd.set_option('display.max_rows', None)
comparison_table

86


/home/rraiyan/personal_projects/tfo_inverse_modelling/inverse_modelling_tfo/data/intensity_interpolation.py:28: RuntimeWarning: invalid value encountered in log
  Y = np.log(data['Intensity'].to_numpy()).reshape(-1, 1)
/home/rraiyan/personal_projects/tfo_inverse_modelling/inverse_modelling_tfo/data/intensity_interpolation.py:28: RuntimeWarning: invalid value encountered in log
  Y = np.log(data['Intensity'].to_numpy()).reshape(-1, 1)
/home/rraiyan/personal_projects/tfo_inverse_modelling/inverse_modelling_tfo/data/intensity_interpolation.py:28: RuntimeWarning: invalid value encountered in log
  Y = np.log(data['Intensity'].to_numpy()).reshape(-1, 1)


This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0


,Sheep,Round,Year,Info,a0_min,a0_max,a1_min,a1_max,a2_min,a2_max,a3_min,a3_max
0,3,1,su2020,,-35.248784,-24.528035,0.364721,0.549395,-28.582966,-20.425855,36.603944,52.664728
1,3,1,su2020,Baseline,-39.309077,-32.390626,0.515026,0.623982,-31.774156,-26.573480,48.378680,58.910716
2,3,2,su2020,,43.470612,111.332836,-1.792847,-0.641300,27.563514,78.281354,-160.092074,-60.241777
3,3,2,su2020,Baseline,68.673163,76.456505,-1.127592,-0.996139,44.994110,50.804348,-106.993765,-95.517313
4,4,1,su2020,,45.643135,73.459823,-1.115784,-0.692435,29.831762,49.505324,-103.859795,-64.276240
5,4,1,su2020,Baseline,44.409257,48.187738,-0.714377,-0.648342,28.342349,31.248000,-67.228586,-61.536095
6,4,2,su2020,,-89.716531,-15.304545,0.136995,1.014466,-59.509271,-12.285610,21.949203,120.879628
7,4,2,su2020,Baseline,-8.783772,10.612249,-0.211044,0.051933,-8.091760,4.970309,-13.431194,13.326116
8,4,3,su2020,,-79.391119,-71.632144,0.675091,0.811431,-50.559025,-44.558396,92.165533,103.918536
9,4,3,su2020,Baseline,-80.239854,-76.473223,0.776147,0.831516,-51.163573,-48.811202,100.327900,104.995056


In [15]:
# Print 4 detector rounds - For cross check!
for tag in all_tags:
    if tag['additional_info'] != '':
        continue
    # Not all rounds have avaialble normalizations constants/ignore those
    try:
        sdd = shd.get_sdd_distance(tag)
        if len(sdd) == 4:
            print(tag)
    except:
        continue

{'experiment_number': 3, 'experiment_round': 1, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'experiment_type': 'sheep'}
{'experiment_number': 3, 'experiment_round': 2, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'experiment_type': 'sheep'}
{'experiment_number': 4, 'experiment_round': 1, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'experiment_type': 'sheep'}
{'experiment_number': 4, 'experiment_round': 2, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'experiment_type': 'sheep'}
{'experiment_number': 4, 'experiment_round': 3, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'experiment_type': 'sheep'}
{'experiment_number': 5, 'experiment_round': 1, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'e

## Rounds with consistent alphas between Baseline and Regular Round
### Summer 2020
3 - 1  
3 - 2  
4 - 1  
4 - 2  
4 - 3  
5 - 1 (Inconsistent)  
5 - 2  

### Spring 2021
1 - 1  
1 - 2  
1 - 3  
2 - 1  
2 - 2  
2 - 3  
  
### Spring 2022
1 - 1  
1 - 2  
1 - 3  
2 - 1  
2 - 2  
2 - 3  
4 - 1  
4 - 2  
7 - 1  
7 - 3  
10 - 1  
10 - 2  
11 - 1  
11 - 2  